# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Creating the blank list
file_path_fin_list = []

# checking your current working directory
print('Current Directory : '+os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Displaying the root of the path
    print('root : '+root)
    
    # Checking all the files in the path provided.
    # Since there is another folder (/home/workspace/event_data/.ipynb_checkpoints) created hence in below steps consolidating the files in parent and child folder
    # Check last 2 files in the below list printed
    file_path_list = glob.glob(os.path.join(root,'*'))
    file_path_fin_list = file_path_fin_list + file_path_list
    
print(file_path_fin_list)

Current Directory : /home/workspace
root : /home/workspace/event_data
root : /home/workspace/event_data/.ipynb_checkpoints
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-event

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_fin_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Checking the number of rows written into consolidated CSV File

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of project. 

## To work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Making a connection to a Cassandra instance in our local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Creating Session for executing query
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating Keyspace with name rahul_db

In [6]:
# This query creates the Keyspace / Database in general terms. 
# Keyspace name is rahul_db

try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS rahul_db
                     WITH REPLICATION = 
                     { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)
except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
# Setting session and connceting to Keyspace

try:
    session.set_keyspace('rahul_db')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

In [8]:
## Creating songs_session table to fetch artist, song title and song's length in the music app history that was heard with -
## sessionId = 338, and itemInSession = 4
## Primary Key has been set as session_id, item_in_session because requirement is to fetch data based upon these 2 columns

try:
    session.execute("DROP TABLE IF EXISTS songs_session")
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS songs_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                 PRIMARY KEY(session_id, item_in_session) \
                 )"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# Extracting Data from consolidated CSV file which we extracted in above steps
# We will read the lines in CSV file one by one and start inserting it into temporary table which we created
# Temporary table in this instance is songs_session

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO songs_session (session_id, item_in_session, artist, song, length)"
            query = query + " VALUES (%s, %s, %s, %s, %s)"
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Fetching records based upon the requirement 
## session_id=338 AND item_in_session=4

try:
    query = "SELECT artist, song, length FROM songs_session WHERE session_id=338 AND item_in_session=4"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('Artist : '+row.artist, ' | Song : '+row.song, ' | Duration : '+str(row.length)+' Seconds')

Artist : Faithless  | Song : Music Matters (Mark Knight Dub)  | Duration : 495.30731201171875 Seconds


### Query 2

In [11]:
## Creating songs_userid table to fetch artist, song, user first name and user last name in the music app history that was heard with -
## userid = 10 and sessionId = 182
## Partition Key has been set as (userid, session_id) together with cluster key item_in_session because requirement -
## is to fetch data based upon these 2 columns and song arranged based upon item_in_session

try:
    session.execute("DROP TABLE IF EXISTS songs_userid")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS songs_userid "
query = query + "(userid int, session_id int, artist text, song text, item_in_session int, first_name text, last_name text, \
                 PRIMARY KEY((userid, session_id), item_in_session) \
                 )"
try:
    session.execute(query)
except Exception as e:
    print(e)
                
                    

In [12]:
# Extracting Data from consolidated CSV file which we extracted in above steps
# We will read the lines in CSV file one by one and start inserting it into temporary table which we created
# Temporary table in this instance is songs_userid

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO songs_userid (userid, session_id, artist, song, item_in_session, first_name, last_name)"
            query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
except Exception as e:
    print(e)
                    

In [13]:
## Fetching records based upon the requirement 
## userid=10 and session_id=182

try:
    query = "SELECT artist, song, first_name, last_name FROM songs_userid WHERE userid=10 AND session_id=182"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('Artist : '+row.artist, ' | Song : '+row.song, ' | User First Name : '+row.first_name, ' | User Last Name : '+row.last_name)

Artist : Down To The Bone  | Song : Keep On Keepin' On  | User First Name : Sylvie  | User Last Name : Cruz
Artist : Three Drives  | Song : Greece 2000  | User First Name : Sylvie  | User Last Name : Cruz
Artist : Sebastien Tellier  | Song : Kilometer  | User First Name : Sylvie  | User Last Name : Cruz
Artist : Lonnie Gordon  | Song : Catch You Baby (Steve Pitron & Max Sanna Radio Edit)  | User First Name : Sylvie  | User Last Name : Cruz


### Query 3

In [14]:
## Creating user_song table to fetch song, user first name and user last name in the music app history that was heard with -
## song = 'All Hands Against His Own'
## Primary Key has been set as song and userid because requirement is to fetch data based upon song and user names are related to userid

try:
    session.execute("DROP TABLE IF EXISTS user_song")
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, userid int, first_name text, last_name text, \
                 PRIMARY KEY(song, userid) \
                 )"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
# Extracting Data from consolidated CSV file which we extracted in above steps
# We will read the lines in CSV file one by one and start inserting it into temporary table which we created
# Temporary table in this instance is user_song

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO user_song (song, userid, first_name, last_name)"
            query = query + " VALUES (%s, %s, %s, %s)"
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
except Exception as e:
    print(e)
                    

In [16]:
## Fetching records based upon the requirement 
## song = 'All Hands Against His Own'

try:
    query = "SELECT first_name, last_name FROM user_song WHERE song='All Hands Against His Own'"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print('User First Name : '+row.first_name, '| User Last Name : '+row.last_name)

User First Name : Jacqueline | User Last Name : Lynch
User First Name : Tegan | User Last Name : Levine
User First Name : Sara | User Last Name : Johnson


### Drop the tables before closing out the sessions

In [17]:
## Droping all the Temporary tables after extracting the data as required

In [18]:
try:
    session.execute("DROP TABLE IF EXISTS songs_session")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS songs_userid")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS user_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()